In [1]:
library(glmnet)
library(SLOPE)
library(bigstep)
library(ggplot2)
library(tidyr)
library(gridExtra)
library(knockoff)

Loading required package: Matrix

Loaded glmnet 4.1-8


Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack




### Computer project

In [2]:
generate.data = function(n = 500, p = 450, k = 5) list(
	X = matrix(data = rnorm(n * p, 0, sd=sqrt(1/n)), nrow = n),
	β = 10 * (1:p %in% seq(1, k))
)

In [3]:
estimate.knockoffs = function(model, k, n = 500, p = 450, iters=100) {
	data = generate.data(n = n, p = p, k = k)
	X = data$X
	β = data$β

	iter = function() {
		y = X %*% β + 2*rnorm(n, 0, 1)

		knockoffs = function(X) create.gaussian(X, rep(0, ncol(X)), diag(ncol(X)))
		coefs = knockoff.filter(X, y, knockoffs, statistic = model, fdr = 0.2)$selected
		ncoefs = length(coefs)
		
		return(c(
			power = sum(coefs <= k) / k,
			fdp = sum(coefs > k) / (ncoefs + (ncoefs == 0))
		))
	}
	suppressWarnings(rowMeans(replicate(iters, iter())))
}

# FDR and power of LASSO, LASSO + knockoffs and ridge

In [4]:
stat_ridge <- function(X, X_k, y) {
  stat.glmnet_coefdiff(X, X_k, y,
    alpha = 0,
    intercept = FALSE,
    standarize = FALSE
  )
}

estimate.knockoffs(stat_ridge, k=5, iters=100)
estimate.knockoffs(stat_ridge, k=20, iters=100)
estimate.knockoffs(stat_ridge, k=50, iters=100)

power       fdp 
0.4060000 0.1232879

power       fdp 
0.7445000 0.1787895

power       fdp 
0.6122000 0.2115327

In [5]:
stat_lasso1 <- function(X, X_k, y) {
  stat.glmnet_coefdiff(X, X_k, y,
    alpha = 1,
    intercept = FALSE,
    standarize = FALSE
  )
}

estimate.knockoffs(stat_lasso1, k=5, iters=100)
estimate.knockoffs(stat_lasso1, k=20, iters=100)
estimate.knockoffs(stat_lasso1, k=50, iters=100)

power       fdp 
0.7820000 0.1581505

power      fdp 
0.968000 0.174691

power       fdp 
0.9576000 0.1863095

In [6]:
estimate.net = function(k, n = 500, p = 450, iters=100, alpha=1) {
	data = generate.data(n = n, p = p, k = k)
	X = data$X
	β = data$β

	iter = function() {
		y = X %*% β + 2*rnorm(n, 0, 1)
		B = coef(cv.glmnet(X, y, alpha=alpha, intercept=F, standarize=F))[-1,]
		coefs = which(B != 0)
		ncoefs = length(coefs)

		return(c(
			E1  = sum((B - β)**2),
			E2  = sum((X %*% (B - β))**2),
			power = sum(coefs <= k) / k,
			fdp = sum(coefs > k) / (ncoefs + (ncoefs == 0))
		))
	}
	suppressWarnings(rowMeans(replicate(iters, iter())))
}

In [7]:
# ridge
estimate.net(alpha=0, k=5, iters=100)
estimate.net(alpha=0, k=20, iters=100)
estimate.net(alpha=0, k=50, iters=100)

E1          E2       power         fdp 
497.2690111 478.1063736   1.0000000   0.9888889

E1           E2        power          fdp 
1454.4181904  995.8892239    1.0000000    0.9555556

E1           E2        power          fdp 
2376.8340587 1287.8475035    1.0000000    0.8888889

In [8]:
# lasso
estimate.net(alpha=1, k=5, iters=100)
estimate.net(alpha=1, k=20, iters=100)
estimate.net(alpha=1, k=50, iters=100)

E1          E2       power         fdp 
249.8492279 234.6325668   0.9140000   0.2053997

E1          E2       power         fdp 
555.9587714 474.7912608   0.9935000   0.5016865

E1           E2        power          fdp 
1124.8513459  780.1526564    0.9938000    0.5286091

In [9]:
estimate.ols = function(k, n = 500, p = 450, iters=100) {
	data = generate.data(n = n, p = p, k = k)
	X = data$X
	β = data$β

	iter = function() {
		y = X %*% β + 2*rnorm(n, 0, 1)
		B = coef(glmnet(X, y, alpha=0, lambda=0, intercept=F, standarize=F))[-1,]

		return(c(
			E1  = sum((B - β)**2),
			E2  = sum((X %*% (B - β))**2)
		))
	}
	suppressWarnings(rowMeans(replicate(iters, iter())))
}

In [10]:
estimate.ols(k=5, iters=100)
estimate.ols(k=20, iters=100)
estimate.ols(k=50, iters=100)

E1        E2 
16577.103  1790.694

E1        E2 
16709.427  1816.236

E1        E2 
15558.641  1779.816